In [ ]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json


In [ ]:
# Read price data from Excel
price_data = pd.read_excel('price_data.xlsx', sheet_name= None)
#price_value = price_data['Price'].values  # Assuming the second column is named 'Price'

# Parameters
Seff = 1
Ceff = 1
Deff = 1
cap_power = 1  # Example value
cap_energy = 4  # Example value
η = 1000
dt = 1

λ_0 = 0.5
λ_min = 0.2
initial_soc = λ_0 * cap_energy


# Model

In [ ]:
def optimize_revenue(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    total_time_period = len(price_vector)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb_e = {t: price_vector['arb_energy_price'].values[t-1] for t in T}
    p_reg_p = {t: price_vector['reg_capacity_price'].values[t-1] for t in T}
    p_reg_e = {t: price_vector['reg_energy_price'].values[t-1] for t in T}
    p_pres_p = {t: price_vector['pres_capacity_price'].values[t-1] for t in T}
    p_pres_e = {t: price_vector['pres_energy_price'].values[t-1] for t in T}
    p_cres_p = {t: price_vector['cres_capacity_price'].values[t-1] for t in T}
    p_cres_e = {t: price_vector['cres_energy_price'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    model.C = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.D = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.C_arb = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.D_arb = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.C_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.D_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.C_pres = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.D_pres = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.C_cres = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.D_cres = Var(T, within=NonNegativeReals, bounds = (0, cap_power))

    model.y_ch = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb_e[t]  * (model.D_arb[t] - model.C_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_regulation_rule(model, t):
        return model.R_reg[t] == p_reg_p[t]  * (model.D_reg[t] + model.C_reg[t]) + p_reg_e[t]  * (model.D_reg[t] - model.C_reg[t]) * dt
    model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == p_pres_p[t]  * (model.D_pres[t] + model.C_pres[t]) + p_pres_e[t]  * (model.D_pres[t] + model.C_pres[t]) * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == p_cres_p[t]  * (model.D_cres[t] + model.C_cres[t]) + p_cres_e[t]  * (model.D_cres[t] + model.C_cres[t]) * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    # Constraints
    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            return model.SOC[t] == model.SOC[t-1] * Seff + (model.C[t] * Ceff - model.D[t]/Deff) * dt
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    if current_period == last_period:

        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == start_soc #* Seff 
        model.SOC_final = Constraint(rule=soc_final_state_constraint)

    def total_charge_rule(model, t):
        return model.C[t] == model.C_arb[t] + model.C_reg[t] + model.C_pres[t] + model.C_cres[t]
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        return model.D[t] == model.D_arb[t] + model.D_reg[t] + model.D_pres[t] + model.D_cres[t]
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    def soc_cap_constraint(model, t):
        return model.SOC[t] >= cap_energy * λ_min
    model.SOC_cap = Constraint(SOC_T, rule=soc_cap_constraint)

    def energy_cap_C_constraint(model):
        return sum(model.C[t] for t in T) <= η * cap_energy
    model.energy_cap_C = Constraint(rule=energy_cap_C_constraint)

    def energy_cap_D_constraint(model):
        return sum(model.D[t] for t in T) <= η * cap_energy
    model.energy_cap_D = Constraint(rule=energy_cap_D_constraint)

    def c_limit_constraint(model, t):
        return model.C[t] <= model.y_ch[t] * cap_power
    model.C_limit = Constraint(T, rule=c_limit_constraint)

    def d_limit_constraint(model, t):
        return model.D[t] <= (1 - model.y_ch[t]) * cap_power
    model.D_limit = Constraint(T, rule=d_limit_constraint)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.C[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.D[t].value for t in T]  # Discharging as positive

    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    # Return the results and final SOC
    return model.obj(), charging_schedule, discharging_schedule, soc_schedule

# Run

In [ ]:
data = price_data

# Iteratively merge all dataframes on 'Timestamp'
price_df = pd.DataFrame({'Time': data[list(data.keys())[0]]['Time']})
for key in data:
    price_df = pd.merge(price_df, data[key], on='Time', how='outer')


In [ ]:
time_period = 24    # periods

total_time_period = len(price_df)

num_slices = total_time_period // time_period

# Lists to store results
total_revenue = 0
all_charging_schedules = []
all_discharging_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices - 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_price = price_df[p*time_period:(p+1)*time_period]
    revenue, charging_schedule, discharging_schedule, soc_schedule = optimize_revenue(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

In [ ]:
# Create a DataFrame for the results
data = {
    't': range(1, len(all_charging_schedules) + 1),
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

In [ ]:
# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['net_discharge'], mode='lines', name='net'))
fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_charge_discharge.show()

# Create interactive plot for charging and discharging
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index, y=result_df['soc']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_soc.show()

In [ ]:
# Create the combined layout
combined_html = f"""
<html>
    <head>
        <title>BESS Revenue Simulation Dashboard</title>
        <style>
            .responsive-plot {{
                width: 80%;
                max-width: 3600px;
                height: 80%;
                max-height: 600px;
                margin: auto;
            }}
        </style>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

    </head>
    <body>
        <h1>BESS Revenue Simulation Dashboard</h1>
        <div>
            <h3>Charging and Discharging Schedules</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_charge_discharge, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
        <div>
            <h3>State of Charge Over Time</h3>
            <div id="plot2" class="responsive-plot">
                {pio.to_html(fig_soc, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
    </body>
</html>
"""

# Save the combined HTML to a file
with open("bess_dashboard.html", "w") as f:
    f.write(combined_html)